In [3]:
import pandas as pd

df = pd.read_csv('./lendingclub_ver3.csv')

In [5]:
df.drop(['Unnamed: 0'], axis = 1,inplace = True)

In [6]:
df

,home_ownership1,home_ownership2,home_ownership3,home_ownership4,home_ownership5,verification_status1,verification_status2,addr_state1,addr_state2,addr_state3,...,delinq_2yrs_log,fico_average_log,revol_bal_log,revol_util_log,total_acc_log,collections_12_mths_ex_med_log,tot_coll_amt_log,purpose_group1,purpose_group2,purpose_group4
0,0,0,0,0,0,0,1,0,0,0,...,0.693147,6.511745,9.272846,0.538830,3.332205,0.000000,0.0,0,1,0
1,0,1,0,0,0,0,1,0,0,0,...,0.000000,6.541030,8.979794,0.521172,1.945910,0.000000,0.0,0,1,0
2,0,0,0,0,0,0,1,0,0,0,...,1.098612,6.504288,8.333751,0.228728,2.708050,0.000000,0.0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0.000000,6.533789,8.956222,0.448525,3.555348,0.000000,0.0,0,1,0
4,0,1,0,0,0,0,1,0,0,0,...,0.000000,6.650279,8.752265,0.148420,2.772589,0.000000,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896882,0,1,0,0,0,0,0,0,0,0,...,0.000000,6.496775,9.062072,0.612479,3.178054,0.000000,0.0,1,0,0
896883,0,0,0,0,0,0,0,0,0,0,...,0.000000,6.511745,9.549523,0.599386,2.772589,0.000000,0.0,0,1,0
896884,0,0,0,0,0,0,1,0,0,0,...,0.693147,6.548219,9.018574,0.266969,3.044522,0.693147,0.0,0,1,0
896885,0,0,0,0,0,0,0,0,0,0,...,0.000000,6.526495,9.308555,0.478096,3.135494,0.000000,0.0,0,1,0


In [7]:
X, y = df.drop('depvar', axis=1), df['depvar']

In [8]:
# train, validation, test set 분리

from sklearn.model_selection import cross_val_score, cross_val_score, train_test_split

X_cv, X_t, y_cv, y_t = train_test_split(X, y, test_size = 0.4)
X_val, X_tst, y_val, y_tst = train_test_split(X_t, y_t, test_size = 0.5)

In [9]:
print(X_cv.shape, X_val.shape, X_tst.shape)

(538132, 86) (179377, 86) (179378, 86)


In [11]:
!pip install imblearn

     |████████████████████████████████| 226 kB 7.4 MB/s eta 0:00:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0


In [12]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority')
X_under, y_under = undersample.fit_resample(X_cv, y_cv)
print(X_under.shape, y_under.shape)


(173818, 86) (173818,)


In [13]:
# undersampling 한한 후후의의 레이블블 확인인
print(sum(y_under == 1))
print(sum(y_under == 0))

86909
86909


In [16]:
# 10Fold 교차검증
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

lr_clf = LogisticRegression()
skfold = StratifiedKFold(n_splits=10)
cv_accuracy = []
cv_precision = []
cv_recall = []
n_iter = 0

for train_index, test_index in skfold.split(X_under, y_under):
  # kfold.split()으로로 반환된된 인덱스를를 이용해해 학습용용, 검증용용 테스트트 데이터터 추출
  X_train, X_test = X_under.iloc[train_index], X_under.iloc[test_index]
  y_train, y_test = y_under.iloc[train_index], y_under.iloc[test_index]
  # 학습습 및및 예측
  lr_clf.fit(X_train, y_train)
  pred = lr_clf.predict(X_test)
  n_iter += 1
  # 반복복 시마다다 정확도도 측정정
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  conf_matrix = confusion_matrix(y_test, pred)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]
  print('\n#{0} 교차차 검증증 정확도: {1}, 정밀도도: {2}, 재현율율: {3}, 학습습 데이터터 크기기: {4}, 검증증 데이터터 크기기: {5}'.format(n_iter, accuracy, precision, recall, train_size, test_size))
  print('#{0} 검증증 세트트 인덱스스: {1}'.format(n_iter, test_index))

  cv_accuracy.append(accuracy)
  cv_precision.append(precision)
  cv_recall.append(recall)

# 개별별 iteration별별 정확도를를 합하여여 평균균 정확도도 계산산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))
print('\n## 평균 검증 정밀도:', np.mean(cv_precision))
print('\n## 평균 검증 재현율:', np.mean(cv_recall))
print(conf_matrix)

/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#1 교차차 검증증 정확도: 0.5984351628121045, 정밀도도: 0.5974706619573886, 재현율율: 0.6033828098032448, 학습습 데이터터 크기기: 156436, 검증증 데이터터 크기기: 17382
#1 검증증 세트트 인덱스스: [    0     1     2 ... 95597 95598 95599]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#2 교차차 검증증 정확도: 0.6025198481187435, 정밀도도: 0.5994863778472532, 재현율율: 0.6177655045449315, 학습습 데이터터 크기기: 156436, 검증증 데이터터 크기기: 17382
#2 검증증 세트트 인덱스스: [  8691   8692   8693 ... 104288 104289 104290]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#8 교차차 검증증 정확도: 0.6005062708549074, 정밀도도: 0.600529404994821, 재현율율: 0.6003912092969739, 학습습 데이터터 크기기: 156436, 검증증 데이터터 크기기: 17382
#8 검증증 세트트 인덱스스: [ 60837  60838  60839 ... 156434 156435 156436]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#9 교차차 검증증 정확도: 0.6037627294171797, 정밀도도: 0.6002446347158902, 재현율율: 0.6211737629459149, 학습습 데이터터 크기기: 156437, 검증증 데이터터 크기기: 17381
#9 검증증 세트트 인덱스스: [ 69528  69529  69530 ... 165124 165125 165126]

#10 교차차 검증증 정확도: 0.600425752258213, 정밀도도: 0.5974112921222942, 재현율율: 0.6160395811759292, 학습습 데이터터 크기기: 156437, 검증증 데이터터 크기기: 17381
#10 검증증 세트트 인덱스스: [ 78219  78220  78221 ... 173815 173816 173817]

## 평균 검증 정확도: 0.6011230249020922

## 평균 검증 정밀도: 0.5989022072076529

## 평균 검증 재현율: 0.6123762647999419
[[5082 3608]
 [3337 5354]]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
from imblearn.over_sampling import RandomOverSampler

X ,y = df.drop(['depvar'],axis = 1),df['depvar']
oversample = RandomOverSampler(sampling_strategy = 'minority')
X_over,y_over= oversample.fit_resample(X,y)
print(X_over.shape, y_over.shape)


(1503024, 86) (1503024,)


In [21]:
# 10Fold 교차검증
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

lr_clf = LogisticRegression()
skfold = StratifiedKFold(n_splits=10)
cv_accuracy = []
cv_precision = []
cv_recall = []
n_iter = 0

for train_index, test_index in skfold.split(X_over, y_over):
  # kfold.split()으로로 반환된된 인덱스를를 이용해해 학습용용, 검증용용 테스트트 데이터터 추출
  X_train, X_test = X_over.iloc[train_index], X_over.iloc[test_index]
  y_train, y_test = y_over.iloc[train_index], y_over.iloc[test_index]
  # 학습습 및및 예측
  lr_clf.fit(X_train, y_train)
  pred = lr_clf.predict(X_test)
  n_iter += 1
  # 반복복 시마다다 정확도도 측정정
  accuracy = accuracy_score(y_test, pred)
  precision = precision_score(y_test, pred)
  recall = recall_score(y_test, pred)
  conf_matrix = confusion_matrix(y_test, pred)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]
  print('\n#{0} 교차차 검증증 정확도: {1}, 정밀도도: {2}, 재현율율: {3}, 학습습 데이터터 크기기: {4}, 검증증 데이터터 크기기: {5}'.format(n_iter, accuracy, precision, recall, train_size, test_size))
  print('#{0} 검증증 세트트 인덱스스: {1}'.format(n_iter, test_index))

  cv_accuracy.append(accuracy)
  cv_precision.append(precision)
  cv_recall.append(recall)

# 개별별 iteration별별 정확도를를 합하여여 평균균 정확도도 계산산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))
print('\n## 평균 검증 정밀도:', np.mean(cv_precision))
print('\n## 평균 검증 재현율:', np.mean(cv_recall))
print(conf_matrix)

/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#1 교차차 검증증 정확도: 0.6075727031396579, 정밀도도: 0.6099684405267167, 재현율율: 0.5966653803675267, 학습습 데이터터 크기기: 1352721, 검증증 데이터터 크기기: 150303
#1 검증증 세트트 인덱스스: [     0      1      2 ... 475627 475634 475637]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#2 교차차 검증증 정확도: 0.6035408474880741, 정밀도도: 0.6000643004115226, 재현율율: 0.6208965948556906, 학습습 데이터터 크기기: 1352721, 검증증 데이터터 크기기: 150303
#2 검증증 세트트 인덱스스: [ 88699  88700  88701 ... 901811 901812 901813]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...


#8 교차차 검증증 정확도: 0.6019281180556479, 정밀도도: 0.600414241518536, 재현율율: 0.6094662745672047, 학습습 데이터터 크기기: 1352722, 검증증 데이터터 크기기: 150302
#8 검증증 세트트 인덱스스: [ 628534  628535  628536 ... 1352719 1352720 1352721]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#9 교차차 검증증 정확도: 0.6118814120903248, 정밀도도: 0.6128015240548446, 재현율율: 0.6078029567138162, 학습습 데이터터 크기기: 1352722, 검증증 데이터터 크기기: 150302
#9 검증증 세트트 인덱스스: [ 716832  716833  716835 ... 1427870 1427871 1427872]


/Users/minjung/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



#10 교차차 검증증 정확도: 0.5944764540724674, 정밀도도: 0.590853252802375, 재현율율: 0.614416308498889, 학습습 데이터터 크기기: 1352722, 검증증 데이터터 크기기: 150302
#10 검증증 세트트 인덱스스: [ 804992  804993  804994 ... 1503021 1503022 1503023]

## 평균 검증 정확도: 0.5945586668178185

## 평균 검증 정밀도: 0.5924223104100761

## 평균 검증 재현율: 0.6106329629633586
[[43177 31974]
 [28977 46174]]
